# S/KEY 해시체인 인증 구조를 Python + FastAPI로 초간단하게 구현한 예시

## ✅ 목적

- 클라이언트는 해시체인의 다음 값을 보내고,
- 서버는 저장된 값의 **한 단계 전 해시값과 일치**하는지 확인하고,
- 일치하면 인증 성공 → 해시체인을 한 칸 내려 저장

---

## ✅ 코드: FastAPI 기반 S/KEY 인증 서버

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import hashlib

app = FastAPI()

# 임시 저장소 (실제론 DB)
server_store = {
    "username": "a81f2c..."  # 서버는 H^n(password)만 저장
}

# 클라이언트 요청 스키마
class AuthRequest(BaseModel):
    username: str
    otp: str

def hash_once(data: str) -> str:
    return hashlib.sha256(data.encode()).hexdigest()

@app.post("/auth")
def authenticate(req: AuthRequest):
    stored_hash = server_store.get(req.username)

    if not stored_hash:
        raise HTTPException(status_code=404, detail="User not found")

    # 사용자가 보낸 OTP를 한 번 더 해시해서 비교
    if hash_once(req.otp) == stored_hash:
        # 인증 성공 → 서버는 다음 단계 값 저장
        server_store[req.username] = req.otp
        return {"status": "success", "message": "Authentication passed"}
    else:
        raise HTTPException(status_code=401, detail="Invalid OTP")



## ✅ 사용 예시

### 1. 서버 초기 저장값 (예: `H^3(password)`) 저장

```python
# 예시 초기값 설정 (클라이언트는 H^2(password) 준비)
server_store = {
    "username": "c9d1ef9db19adf8b231..."  # <- H^3(P)
}

```

### 2. 클라이언트는 H^2(P)를 보내고,

서버는 `H(H^2(P)) == H^3(P)`를 비교하여 통과 여부 판단

---

## ✅ 실행 방법

1. `uvicorn main:app --reload`
2. Postman 또는 `curl`로 테스트

```bash
curl -X POST http://localhost:8000/auth \
 -H "Content-Type: application/json" \
 -d '{"username": "username", "otp": "<사용자 측 OTP>"}'
```

---

## ✅ 장점

- 외부 라이브러리 없음 (`hashlib`, `fastapi`, `pydantic`만)
- 암호 저장 필요 없음
- 비동기식 → 시간 동기화 X

---

## ✅ 한줄 요약

> 이 코드는 "서버는 현재 해시값(H^n)을 저장하고, 클라이언트는 H^(n-1)을 보내 인증하는 S/KEY 구조를 구현"한 인증 API입니다.
>